In [1]:
import Homework1

# Task 2

In [2]:
EASY = [
    {
        "question": "I ate 2 pears and 1 apple. How many fruit did I eat?",
        "answer": 3
    },
    {
        "question": "I had ten chocolates but ate one. How many remain?",
        "answer": 9
    }
]

Homework1.accuracy_zero_shot(EASY)

  0%|          | 0/5 [00:00<?, ?it/s]

[1.0, 1.0, 1.0, 1.0, 1.0]

# Task 3

In [3]:
import datasets

TEST = datasets.load_dataset("nuprl/llm-systems-math-word-problems", split="test")
print(Homework1.accuracy_zero_shot(TEST))

  0%|          | 0/5 [00:00<?, ?it/s]

[0.16, 0.18, 0.14, 0.16, 0.14]


# Task 4

In [4]:
Homework1.accuracy_few_shot(TEST)

  0%|          | 0/5 [00:00<?, ?it/s]

[0.16, 0.16, 0.18, 0.18, 0.16]